In [3]:
# install verdict
!uv pip install verdict --system

# This notebook has been run ahead of time, so you can inspect outputs without making
# any API calls. You can set your API key if you want to run the examples yourself.
# %env OPENAI_API_KEY=*************************

#### ExpertQA Hallucination Detection
#### See https://verdict.haizelabs.com/docs for a walkthrough of this example!

In [4]:
from datasets import load_dataset

from verdict import Pipeline, Layer
from verdict.common.judge import CategoricalJudgeUnit
from verdict.scale import DiscreteScale
from verdict.transform import MaxPoolUnit
from verdict.dataset import DatasetWrapper

# We'll disable Verdict rate-limiting for this example. By default, Verdict follows the OpenAI Tier 1 rate limits for `gpt-4o-mini`.
from verdict.util import ratelimit
ratelimit.disable()

#   1. A CategoricalJudgeUnit first decides if a hallucination is present...
#   2. Followed by a CategoricalJudgeUnit that verifies the initial judge's explanation and decision.

judge_then_verify = CategoricalJudgeUnit(name='Judge', categories=DiscreteScale(['yes', 'no']), explanation=True).prompt("""
    Determine whether the provided claim is consistent with the corresponding document. Consistency in this context implies that
    all information presented in the claim is substantiated by the document. If not, it should be considered inconsistent.
    Document: {source.doc}
    Claim: {source.claim}
    Please assess the claim’s consistency with the document by responding with either "yes" or "no".
    Answer:
""").via(policy_or_name='gpt-4o', retries=3, temperature=0.7) \
\
>> CategoricalJudgeUnit(name='Verifier', categories=DiscreteScale(['yes', 'no'])).prompt("""
    Check if the given answer correctly reflects whether the claim is consistent with the corresponding document. Consistency in this context implies that
    all information presented in the claim is substantiated by the document. If not, it should be considered inconsistent. "yes" means the claim is consistent with the document, and "no" means the claim is not consistent with the document.
    
    Document: {source.doc}
    Claim: {source.claim}
    Answer: {previous.choice}
    Answer Justification: {previous.explanation}
    
    If you think the answer is correct, return the answer as is. If it's incorrect, return the opposite (if answer = "yes", return "no", and if answer = "no", return "yes").
    """).via(policy_or_name='gpt-4o', retries=3, temperature=0.0)

# ...and then aggregate the results of the three judges+verifiers with a `MaxPoolUnit`
pipeline = Pipeline() \
    >> Layer(judge_then_verify, repeat=3) \
    >> MaxPoolUnit()

# Load the ExpertQA dataset (https://arxiv.org/pdf/2309.07852)
dataset = DatasetWrapper.from_hf(
    load_dataset('lytang/LLM-AggreFact').filter(
        lambda row: row['dataset'] == 'ExpertQA'
    ),
    columns=['claim', 'doc', 'label'] # we can also specify a custom input_fn to map each row into a Schema
)

# all responses from intermediate Units are available as columns in response_df!
response_df, _ = pipeline.run_from_dataset(dataset['test'], max_workers=1024)
response_df['pred_label'] = response_df['Pipeline_root.block.block.unit[Map MaxPool]_choice'] == 'yes'

from verdict.util.experiment import display_stats, ExperimentConfig
display_stats(response_df, ExperimentConfig(ground_truth_cols=['label'], prediction_cols=['pred_label']));

───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Ground Truth   Prediction             Acc.              Cohen (κ)           Kendall (τ)         Spearman (ρ)     
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
         label   pred_label            78.88%                0.02                0.03                 0.03         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────